# Car Price Prediction using Linear Regression

In [1]:
import pandas as pd
import numpy as np

In [2]:
car = pd.read_csv('sample_data/car_data.csv')
car.head()


,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [3]:
car.shape

(892, 6)

In [4]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [5]:
backup=car.copy()

***Data Cleaning***

In [6]:
car=car[car['year'].str.isnumeric()]

In [7]:
car['year']=car['year'].astype(int)

In [8]:
car=car[car['Price']!='Ask For Price']

In [9]:
car['Price'] = car['Price'].str.replace(',', '').astype(int)

In [10]:
car['kms_driven']=car['kms_driven'].str.split().str.get(0).str.replace(',','')

In [11]:
car=car[car['kms_driven'].str.isnumeric()]

In [12]:
car['kms_driven']=car['kms_driven'].astype(int)

In [13]:
car=car[~car['fuel_type'].isna()]

In [14]:
car['name']=car['name'].str.split().str.slice(start=0,stop=3).str.join(' ')

In [15]:
car=car.reset_index(drop=True)

***Cleaned Data***

In [16]:
car

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
811,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
812,Tata Indica V2,Tata,2009,110000,30000,Diesel
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
814,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [17]:
car.describe()

,year,Price,kms_driven
count,816.000000,8.160000e+02,816.000000
mean,2012.444853,4.117176e+05,46275.531863
std,4.002992,4.751844e+05,34297.428044
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.912500e+05,56818.500000
max,2019.000000,8.500003e+06,400000.000000


In [18]:
car=car[car['Price']<6e6].reset_index(drop=True)

In [19]:
car.to_csv('Cleaned_Car_data.csv')

***Model***

In [20]:
x=car.drop(columns='Price')
y=car['Price']

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [23]:
ohe=OneHotEncoder()
ohe.fit(x[['name', 'company', 'fuel_type']])

OneHotEncoder()

In [24]:
ohe.categories_

[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
        'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
        'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
        'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat Diesel',
        'Chevrolet Beat LS', 'Chevrolet Beat LT', 'Chevrolet Beat PS',
        'Chevrolet Cruze LTZ', 'Chevrolet Enjoy', 'Chevrolet Enjoy 1.4',
        'Chevrolet Sail 1.2', 'Chevrolet Sail UVA', 'Chevrolet Spark',
        'Chevrolet Spark 1.0', 'Chevrolet Spark LS', 'Chevrolet Spark LT',
        'Chevrolet Tavera LS', 'Chevrolet Tavera Neo', 'Datsun GO T',
        'Datsun Go Plus', 'Datsun Redi GO', 'Fiat Linea Emotion',
        'Fiat Petra ELX', 'Fiat Punto Emotion', 'Force Motors Force',
        'Force Motors One', 'Ford EcoSport', 'Ford EcoSport Ambiente',
        'Ford EcoSport Titanium', 'Ford EcoSport Trend',
        'Ford Endeavor 4x4', 'Ford Fiesta', 'Ford Fiesta SXi', 'Ford Figo',
        '

In [25]:
column_trans= make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name', 'company', 'fuel_type']), remainder='passthrough')

In [26]:
lr=LinearRegression()

In [27]:
pipe=make_pipeline(column_trans, lr)

In [28]:
pipe.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [29]:
y_pred= pipe.predict(X_test)

In [30]:
y_pred

array([  30440.35738657,  397737.91469386,  401652.32190272,
        477784.22518418,  396981.19980691,  450392.35444909,
         98593.00172313,  243717.89993121,  459563.39801189,
        199790.94345015,  382218.89248398,  328578.08285185,
        691210.82161498,  192192.11619774, 2036027.84651541,
        365034.49867972,  612807.6084374 ,  349745.00139212,
        943114.85502959,  249105.61993209,  370986.03970408,
        121728.68282225,  458812.65150424,  603203.00582312,
        359266.99225536,  355283.5277335 ,  458706.78300229,
        732880.87168746,  368355.95545727,  605483.67984141,
        692224.32940031,  407499.81877648,  174984.50318595,
        128485.40139112,  577955.39548885,  193108.81797225,
        382392.37002616,  634667.88360833,  486751.65430399,
        657033.50606208,  254861.31532914,  469299.57007867,
        -50631.22996122,  532637.22377499,  301305.72116377,
        252316.89689256,  412468.25894988, -124008.63057721,
        127349.23895247,

In [31]:
r2_score(y_test, y_pred)

0.5731311949541246

In [32]:
scores=[]
for i in range(1000):
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
  lr=LinearRegression()
  pipe=make_pipeline(column_trans, lr)
  pipe.fit(X_train, y_train)
  y_pred= pipe.predict(X_test)
  scores.append(r2_score(y_test, y_pred))

In [33]:
np.argmax(scores)

np.int64(433)

In [34]:
scores[np.argmax(scores)]

0.8457059012561223

In [35]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans, lr)
pipe.fit(X_train, y_train)
y_pred= pipe.predict(X_test)
r2_score(y_test, y_pred)

0.8457059012561223

In [36]:
import pickle

In [37]:
pickle.dump(pipe, open('LinearRegressionModel.pkl', 'wb'))

In [41]:
pipe.predict(pd.DataFrame([['Audi A8', 'Audi', 2019, 100, 'Petrol']], columns=['name', 'company', 'year', 'kms_driven', 'fuel_type']))

array([1709228.42328179])